In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import sys, os, numpy as np, pandas as pd

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
import matplotlib.pyplot as plt
from pylab import cm
from matplotlib.colors import LogNorm
import matplotlib

plt.rc('axes', labelsize=16)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('legend',fontsize=16)

### Dataset

In [3]:
from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn import tree

In [4]:
# We'll load in the standard iris dataset from scikit-learn
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    test_size=0.5, random_state=42)

### First - decision trees - this is how sklearn performs on the data, we'll try to replicate it

In [5]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

print(f"Accuracy: {100*np.sum(clf.predict(X_test)==y_test)/len(y_test):.1f}%")

Accuracy: 94.7%


# Single-parameter Decision Tree

In [6]:
# Get data with only the 0th parameter
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    test_size=0.5, random_state=42)
X_train = X_train[:,0]
X_test = X_test[:,0]

In [7]:
def train_layer(X, y, idxs, layer=0, max_layers=1):
    
    # Exit if only one unique parameter or target or if we've reached the maximum number of layers
    if (len(np.unique(X))==1) | (len(np.unique(y))==1) | (layer==max_layers):
        # Return the class with the most entries in this subset
        targets, counts = np.unique(y, return_counts=True)
        return targets[np.argmax(counts)], np.nan
    
    
    # Set cut and select subset - here we're just going to use the mean of the
    # parameter value as a cut for simplicity
    cut = np.mean(X)
    subset = X>cut
    
    no = train_layer(X[~subset], y[~subset], idxs[~subset], 
                    layer=layer+1, max_layers=max_layers)
    yes = train_layer(X[subset], y[subset], idxs[subset], 
                    layer=layer+1, max_layers=max_layers)
    
    tree = {'no':no[0], 'yes':yes[0]}
    cuts = {cut:[no[1], yes[1]]}
    
    return tree, cuts

In [8]:
# Build the tree
target_tree, cut_tree = train_layer(X_train, y_train, np.arange(len(y_train)), layer=0, max_layers=2)
target_tree, cut_tree

({'no': {'no': 0, 'yes': 1}, 'yes': {'no': 2, 'yes': 2}},
 {5.848: [{5.137837837837838: [nan, nan]}, {6.5394736842105265: [nan, nan]}]})

In [9]:
def predict_layer(X, values, tree, cuts, target_type=int):
    
    # Return target value if we've reached that in the tree
    if isinstance(tree, target_type):
        return tree
    
    # Get cut from the cut tree
    cut = list(cuts.keys())[0]
    subset = X>cut
    
    # Get target values from next layer down in tree
    values[~subset]=predict_layer(X[~subset], values[~subset], 
                                     tree['no'], cuts[cut][0], 
                                     target_type=target_type)
    values[subset]=predict_layer(X[subset], values[subset], 
                                     tree['yes'], cuts[cut][1], 
                                     target_type=target_type)
    
    return values

In [10]:
values = np.zeros(len(y_test), dtype=int)-1
y_predict = predict_layer(X_test, values, target_tree, cut_tree, target_type=type(y_train[0]))

In [11]:
y_predict

array([2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 0, 0, 2, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 1, 0, 1, 1, 2, 2, 1,
       1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0,
       1, 2, 0, 2, 1, 1, 2, 1, 2])

In [12]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0,
       1, 2, 0, 1, 2, 0, 2, 2, 1])

In [13]:
print(f"Accuracy: {100*np.sum(y_predict==y_test)/len(y_test):.1f}%")

Accuracy: 65.3%


### Add more layers

In [14]:
tree, cuts = train_layer(X_train, y_train, np.arange(len(y_train)), layer=0, max_layers=6)

values = np.zeros(len(y_test), dtype=int)-1
y_predict = predict_layer(X_test, values, tree, cuts, target_type=type(y_train[0]))

print(f"Accuracy: {100*np.sum(y_predict==y_test)/len(y_test):.1f}%")

Accuracy: 70.7%


# Choose which parameter to decide on

In [15]:
# This time we'll use all four parameters in the dataset
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    test_size=0.5, random_state=42)

In [16]:
def train_layer(X, y, layer=0, max_layers=1):
    
    # Exit if only one unique parameter for all parameters or only one unique target
    # or if we've reached the maximum number of layers
    n_unique = np.array([len(np.unique(X[:,i])) for i in range(X.shape[1])])
    if (np.sum(n_unique>1)==0) | (len(np.unique(y))==1) | (layer==max_layers):
        # Return the class with the most entries in this subset
        targets, counts = np.unique(y, return_counts=True)
        return targets[np.argmax(counts)], np.nan, -1
    
    # Choose which parameter to cut on
    param = np.random.choice(np.arange(X.shape[1])[n_unique>1], size=1)[0]
    # Select cut
    cut = np.mean(X[:,param])
    subset = X[:,param]>cut
    
    # Go to next layer of tree
    no = train_layer(X[~subset], y[~subset], layer=layer+1, max_layers=max_layers)
    yes = train_layer(X[subset], y[subset], layer=layer+1, max_layers=max_layers)
    
    tree = {'no':no[0], 'yes':yes[0]}
    cuts = {cut:[no[1], yes[1]]}
    params = {param:[no[2], yes[2]]}
    
    return tree, cuts, params

In [17]:
target_tree, cut_tree, param_tree = train_layer(X_train, y_train, 
                                                layer=0, max_layers=4)

In [18]:
def predict_layer(X, values, tree, cuts, params, target_type=int):
    
    # Return target value if we've reached that in the tree
    if isinstance(tree, target_type):
        return tree
    
    # Get parameter and cut from the cut tree
    param = list(params.keys())[0]
    cut = list(cuts.keys())[0]
    subset = X[:,param]>cut
    
    # Get target values from next layer down in tree
    values[~subset]=predict_layer(X[~subset], values[~subset], 
                                     tree['no'], cuts[cut][0], params[param][0],
                                     target_type=target_type)
    values[subset]=predict_layer(X[subset], values[subset], 
                                     tree['yes'], cuts[cut][1], params[param][1],
                                     target_type=target_type)
    
    return values

In [19]:
values = np.zeros(len(y_test), dtype=int)-1
y_predict = predict_layer(X_test, values, target_tree, cut_tree, param_tree, target_type=type(y_train[0]))

print(f"Accuracy: {100*np.sum(y_predict==y_test)/len(y_test):.1f}%")

Accuracy: 90.7%


### More layers

In [20]:
target_tree, cut_tree, param_tree = train_layer(X_train, y_train, layer=0, max_layers=10)

values = np.zeros(len(y_test), dtype=int)-1
y_predict = predict_layer(X_test, values, target_tree, cut_tree, param_tree, target_type=type(y_train[0]))

print(f"Accuracy: {100*np.sum(y_predict==y_test)/len(y_test)}%")

Accuracy: 90.66666666666667%


# Extend to a forest

A forest is a set of decision trees aggregated together to obtain a combined prediction.

Each tree is different because we use bootstrap resampling to run on a slightly different sample and there are random processes happening in the tree too.

In [21]:
def train_forest(X, y, forest_size=1, max_layers=1):
    
    forest = []
    for i in range(forest_size):

        # Bootstrap (random sample with replacement)
        train_set = np.random.choice(np.arange(X.shape[0]), size=X.shape[0], replace=True)

        # Train tree
        target_tree, cut_tree, param_tree = train_layer(X[train_set], y[train_set], 
                                                        layer=0, max_layers=max_layers)

        # Add to forest
        forest += [(target_tree, cut_tree, param_tree),]
        
    return forest

In [22]:
forest = train_forest(X_train, y_train, forest_size=10, max_layers=10)

In [23]:
def predict(X, forest):
    
    predictions = np.zeros((len(forest), X.shape[0]), dtype=int)
    for i, tree in enumerate(forest):

        # Bootstrap (random sample with replacement)
        predictions[i] = predict_layer(X, values, tree[0], tree[1], tree[2], target_type=type(y_train[0]))
    
    # Aggregate predictions and take modal class in each cast
    classes = np.unique(predictions)
    aggregated = np.array([np.sum(predictions==i, axis=0) for i in classes])
    y_predict = classes[np.argmax(aggregated, axis=0)]

    return y_predict

In [24]:
y_predict = predict(X_test, forest)

In [25]:
print(f"Accuracy: {100*np.sum(y_predict==y_test)/len(y_test)}%")

Accuracy: 98.66666666666667%


### How are we doing compared to sklearn?

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

print(f"Accuracy: {100*np.sum(clf.predict(X_test)==y_test)/len(y_test):.1f}%")

Accuracy: 98.7%


Awesome, we've built a comparable classifier. 

Some details to do with how the tree is build - cut choice, tree aggregation etc - may be different but the principles of how to build a random forest classifier are the same.